In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as ss
import pickle
import tensorflow as tf
from scipy import sparse
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import kneighbors_graph
from skimage.transform import rotate
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras import layers, models
from google.colab import drive
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def load_cifar10():
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()
    return (X_train, y_train), (X_test, y_test)

In [ ]:
(X_train, y_train), (X_test, y_test) = load_cifar10()
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0


170498071/170498071 [==============================] - 2s 0us/step


In [ ]:
# Base imports for model creation and compilation
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Concatenate, Add, GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, Reshape, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

# For functional API usage
from tensorflow.keras import backend as K

# Additional utilities
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.datasets import cifar10

# Make sure to use the correct imports as required for specific layers or utilities


def create_vgg_like_model(input_shape):
    model = Sequential()

    # Block 1
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Block 2
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Block 3
    model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


def create_inception_like_model(input_shape):
    inputs = Input(shape=input_shape)
    # Tower 1
    tower_1 = Conv2D(64, (1, 1), padding='same', activation='relu')(inputs)
    tower_1 = Conv2D(64, (3, 3), padding='same', activation='relu')(tower_1)

    # Tower 2
    tower_2 = Conv2D(64, (1, 1), padding='same', activation='relu')(inputs)
    tower_2 = Conv2D(64, (5, 5), padding='same', activation='relu')(tower_2)

    # Tower 3
    tower_3 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(inputs)
    tower_3 = Conv2D(64, (1, 1), padding='same', activation='relu')(tower_3)

    output = Concatenate(axis=-1)([tower_1, tower_2, tower_3])
    output = Flatten()(output)
    output = Dense(256, activation='relu')(output)
    output = Dropout(0.4)(output)
    output = Dense(10, activation='softmax')(output)

    model = Model(inputs=inputs, outputs=output)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def create_mobilenet_like_model(input_shape):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), strides=(2, 2), padding='same', activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())

    model.add(SeparableConv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(SeparableConv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def create_shallow_wide_model(input_shape):
    model = Sequential([
        Conv2D(256, (3, 3), padding='same', activation='relu', input_shape=input_shape),
        Conv2D(256, (3, 3), padding='same', activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def create_asymmetric_convolution_model(input_shape):
    model = Sequential([
        Conv2D(64, (1, 3), padding='same', activation='relu', input_shape=input_shape),
        Conv2D(64, (3, 1), padding='same', activation='relu'),
        Conv2D(128, (1, 3), padding='same', activation='relu'),
        Conv2D(128, (3, 1), padding='same', activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def create_capsule_inspired_model(input_shape):
    model = Sequential([
        Conv2D(64, (3, 3), activation='relu', input_shape=input_shape),
        Conv2D(64, (3, 3), activation='relu'),
        Flatten(),
        Dense(160, activation='sigmoid'),  # Capsule-like dense layer
        Reshape((10, 16)),  # 10 capsules, 16 dimensions each
        Lambda(lambda x: K.sqrt(K.sum(K.square(x), axis=2))),  # Length of the vector as a capsule's output
        Activation('softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def create_minimalistic_model(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def create_high_dropout_model(input_shape):
    model = Sequential([
        Conv2D(128, (3, 3), padding='same', activation='relu', input_shape=input_shape),
        Dropout(0.5),
        Conv2D(128, (3, 3), padding='same', activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.5),
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def create_increased_bn_model(input_shape):
    model = Sequential([
        Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=input_shape),
        BatchNormalization(),
        Conv2D(64, (3, 3), padding='same', activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),
        Flatten(),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

from tensorflow.keras.layers import Input, Conv2D ,Concatenate,BatchNormalization, ReLU, Add, MaxPooling2D, GlobalAveragePooling2D, Dense, AveragePooling2D

def resnet_lite_block(input_tensor, filters, kernel_size, strides=(1, 1), activation='relu', l2_reg=0.001):
    """A simplified ResNet block."""
    x = Conv2D(filters, kernel_size, strides=strides, padding='same', kernel_regularizer=l2(l2_reg))(input_tensor)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(filters, kernel_size, strides=(1, 1), padding='same', kernel_regularizer=l2(l2_reg))(x)
    x = BatchNormalization()(x)

    shortcut = Conv2D(filters, (1, 1), strides=strides, padding='same', kernel_regularizer=l2(l2_reg))(input_tensor)
    shortcut = BatchNormalization()(shortcut)

    x = Add()([x, shortcut])
    x = ReLU()(x)
    return x

def create_extreme_resnet_model(input_shape):
    inputs = Input(shape=input_shape)
    x = inputs

    # Add multiple residual blocks
    for _ in range(5):  # Extreme ResNet with 5 blocks
        x = resnet_lite_block(x, 64, (3, 3), l2_reg=0.001)
    for _ in range(5):
        x = resnet_lite_block(x, 128, (3, 3), l2_reg=0.001)

    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(10, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=x)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


In [ ]:

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Load CIFAR-10 data
def load_cifar10():
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()
    return (X_train, y_train), (X_test, y_test)

(X_train, y_train), (X_test, y_test) = load_cifar10()
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Define a dictionary of models
models = {
    "vgg_like": create_vgg_like_model((32, 32, 3)),
    "inception_like": create_inception_like_model((32, 32, 3)),
    "mobilenet_like": create_mobilenet_like_model((32, 32, 3)),
    "shallow_wide": create_shallow_wide_model((32, 32, 3)),
    "asymmetric_conv": create_asymmetric_convolution_model((32, 32, 3)),
    "capsule_inspired": create_capsule_inspired_model((32, 32, 3)),
    "minimalistic": create_minimalistic_model((32, 32, 3)),
    "high_dropout": create_high_dropout_model((32, 32, 3)),
    "increased_bn": create_increased_bn_model((32, 32, 3)),
    "extreme_resnet": create_extreme_resnet_model((32, 32, 3))
}



# Training settings
batch_size = 64
epochs = 30

# Loop to compile, train, and save each model
for model_name, model in models.items():
    print(f"Training model: {model_name}")
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Early stopping and model checkpoint
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    model_checkpoint = ModelCheckpoint(f'/content/drive/My Drive/ood_generalization_proj/cnn_models/checkpoints/{model_name}_best.h5', save_best_only=True)

    # Fit the model
    history = model.fit(
        X_train, y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(X_test, y_test),
        callbacks=[early_stopping, model_checkpoint]
    )

    # Optionally, you can save the final model (not just the best checkpoint)
    model.save(f'/content/drive/My Drive/ood_generalization_proj/cnn_models/{model_name}.keras')


Training model: vgg_like
Epoch 1/20
781/782 [============================>.] - ETA: 0s - loss: 1.7656 - accuracy: 0.3323

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


782/782 [==============================] - 83s 105ms/step - loss: 1.7655 - accuracy: 0.3323 - val_loss: 1.4620 - val_accuracy: 0.4659
Epoch 2/20
782/782 [==============================] - 80s 102ms/step - loss: 1.2836 - accuracy: 0.5375 - val_loss: 1.0739 - val_accuracy: 0.6213
Epoch 3/20
782/782 [==============================] - 79s 101ms/step - loss: 1.0258 - accuracy: 0.6387 - val_loss: 0.9025 - val_accuracy: 0.6847
Epoch 4/20
782/782 [==============================] - 80s 102ms/step - loss: 0.8517 - accuracy: 0.7054 - val_loss: 0.8372 - val_accuracy: 0.7146
Epoch 5/20
782/782 [==============================] - 80s 103ms/step - loss: 0.7328 - accuracy: 0.7496 - val_loss: 0.7353 - val_accuracy: 0.7502
Epoch 6/20
782/782 [==============================] - 79s 102ms/step - loss: 0.6379 - accuracy: 0.7832 - val_loss: 0.7460 - val_accuracy: 0.7490
Epoch 7/20
782/782 [==============================] - 80s 102ms/step - loss: 0.5561 - accuracy: 0.8093 - val_loss: 0.6955 - val_accuracy: 0.7

In [ ]:
def create_large_kernel_model(input_shape):
    model = Sequential([
        Conv2D(32, (5, 5), activation='relu', input_shape=input_shape, padding='same'),
        MaxPooling2D((2, 2)),
        Conv2D(64, (5, 5), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def create_multi_scale_features_model(input_shape):
    inputs = Input(shape=input_shape)
    branch1 = Conv2D(32, (1, 1), padding='same', activation='relu')(inputs)
    branch2 = Conv2D(32, (3, 3), padding='same', activation='relu')(inputs)
    branch3 = Conv2D(32, (5, 5), padding='same', activation='relu')(inputs)
    output = Concatenate()([branch1, branch2, branch3])
    output = Flatten()(output)
    output = Dense(256, activation='relu')(output)
    output = Dropout(0.5)(output)
    output = Dense(10, activation='softmax')(output)

    model = Model(inputs=inputs, outputs=output)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

from tensorflow.keras.layers import Conv2D, BatchNormalization, Add, Input, Flatten, Dense, Dropout, Activation
from tensorflow.keras.models import Model

def create_skip_connection_model(input_shape):
    inputs = Input(shape=input_shape)
    # Adjust input tensor channels to match subsequent layers
    adjusted_input = Conv2D(64, (1, 1), padding='same', activation='relu')(inputs)

    # Main convolutional path
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)

    # Skip connection adding the adjusted input with the result of the main path
    skip = Add()([adjusted_input, x])
    skip = Activation('relu')(skip)  # Optional activation layer after adding

    skip = Flatten()(skip)
    skip = Dense(256, activation='relu')(skip)
    skip = Dropout(0.5)(skip)
    output = Dense(10, activation='softmax')(skip)

    model = Model(inputs=inputs, outputs=output)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


def create_dual_path_network_model(input_shape):
    inputs = Input(shape=input_shape)

    # Path 1
    path1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    path1 = Conv2D(64, (3, 3), activation='relu', padding='same')(path1)

    # Path 2
    path2 = SeparableConv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    path2 = SeparableConv2D(64, (3, 3), activation='relu', padding='same')(path2)

    combined = Concatenate()([path1, path2])
    combined = Flatten()(combined)
    combined = Dense(256, activation='relu')(combined)
    combined = Dropout(0.5)(combined)
    output = Dense(10, activation='softmax')(combined)

    model = Model(inputs=inputs, outputs=output)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def create_dense_prediction_model(input_shape):
    model = Sequential([
        Conv2D(64, (3, 3), activation='relu', input_shape=input_shape, padding='same'),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Dense(128, activation='relu'),
        Dense(128, activation='relu'),
        Flatten(),
        Dense(256, activation='relu'),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model



In [ ]:
models = {
    "large_kernel_model": create_large_kernel_model((32, 32, 3)),
    "multi_scale_features_model": create_multi_scale_features_model((32, 32, 3)),
    "skip_connection_model": create_skip_connection_model((32, 32, 3)),
    "dual_path_network_model": create_dual_path_network_model((32, 32, 3)),
    "dense_prediction_model": create_dense_prediction_model((32, 32, 3))
}

batch_size = 64
epochs = 20

# Directory for saving models and checkpoints
model_dir = '/content/drive/My Drive/ood_generalization_proj/cnn_models'
checkpoint_dir = f'{model_dir}/checkpoints'

# Loop to compile, train, and save each model
for model_name, model in models.items():
    print(f"Training model: {model_name}")
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Early stopping and model checkpoint
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    model_checkpoint = ModelCheckpoint(f'{checkpoint_dir}/{model_name}_best.h5', save_best_only=True)

    # Fit the model
    history = model.fit(
        X_train, y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(X_test, y_test),
        callbacks=[early_stopping, model_checkpoint]
    )

    # Save the final model
    model.save(f'{model_dir}/{model_name}.keras')

Training model: large_kernel_model
Epoch 1/20
782/782 [==============================] - 17s 21ms/step - loss: 1.5528 - accuracy: 0.4382 - val_loss: 1.2495 - val_accuracy: 0.5498
Epoch 2/20
782/782 [==============================] - 17s 21ms/step - loss: 1.2200 - accuracy: 0.5651 - val_loss: 1.1250 - val_accuracy: 0.5991
Epoch 3/20
782/782 [==============================] - 17s 21ms/step - loss: 1.0719 - accuracy: 0.6209 - val_loss: 0.9477 - val_accuracy: 0.6664
Epoch 4/20
782/782 [==============================] - 17s 21ms/step - loss: 0.9728 - accuracy: 0.6578 - val_loss: 0.9227 - val_accuracy: 0.6767
Epoch 5/20
782/782 [==============================] - 17s 21ms/step - loss: 0.8856 - accuracy: 0.6867 - val_loss: 0.8804 - val_accuracy: 0.6950
Epoch 6/20
782/782 [==============================] - 17s 21ms/step - loss: 0.8144 - accuracy: 0.7138 - val_loss: 0.8499 - val_accuracy: 0.7033
Epoch 7/20
782/782 [==============================] - 16s 21ms/step - loss: 0.7511 - accuracy: 0.7342

In [ ]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras import datasets, layers, models
from keras.layers import Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Flatten
from keras.regularizers import l2
import matplotlib.pyplot as plt
import numpy as np

# Load dataset
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Standardize images
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

# Flatten the label arrays
train_labels = train_labels.flatten()
test_labels = test_labels.flatten()

# Model architecture with added residual connections and data augmentation
model = Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),

    layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)),
    layers.BatchNormalization(),
    layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.3),

    layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.5),

    layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.5),

    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')  # 10 classes
])

# Compile model with the sparse categorical crossentropy loss
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_images, train_labels, batch_size=64, epochs=30, validation_data=(test_images, test_labels))

model.save(f'{model_dir}/augment_triple_depth.keras')

Epoch 1/30
782/782 [==============================] - 52s 61ms/step - loss: 3.0483 - accuracy: 0.3224 - val_loss: 1.8665 - val_accuracy: 0.4336
Epoch 2/30
782/782 [==============================] - 47s 60ms/step - loss: 1.8669 - accuracy: 0.4220 - val_loss: 1.8150 - val_accuracy: 0.4233
Epoch 3/30
782/782 [==============================] - 47s 60ms/step - loss: 1.7298 - accuracy: 0.4594 - val_loss: 1.9371 - val_accuracy: 0.3938
Epoch 4/30
782/782 [==============================] - 47s 60ms/step - loss: 1.6369 - accuracy: 0.4947 - val_loss: 1.5378 - val_accuracy: 0.5361
Epoch 5/30
782/782 [==============================] - 47s 60ms/step - loss: 1.5955 - accuracy: 0.5204 - val_loss: 1.4492 - val_accuracy: 0.5784
Epoch 6/30
782/782 [==============================] - 47s 60ms/step - loss: 1.5460 - accuracy: 0.5443 - val_loss: 1.4586 - val_accuracy: 0.5787
Epoch 7/30
782/782 [==============================] - 47s 60ms/step - loss: 1.5155 - accuracy: 0.5577 - val_loss: 1.6652 - val_accuracy: